## Ejercicio 1. 
### Creación de una bases de datos de películas extraídas de un API

#### Fase 1: Extracción de datos de películas

In [16]:
import requests
import pandas as pd

# URL de API
url = 'https://beta.adalab.es/resources/apis/pelis/pelis.json'

# Hacer peticion
response = requests.get(url)

# Convertir a formato JSON
datos = response.json()

# Ver cuantas pelis tenemos
print(f"Total de películas en la API: {len(datos)}")



Total de películas en la API: 100


In [17]:
# Convertir a un DataFrame 
df_peliculas = pd.DataFrame(datos)
print(df_peliculas.head(20))

    id                    titulo   año  duracion           genero  adultos  \
0    1             The Godfather  1972       175           Crimen    False   
1    2     The Godfather Part II  1974       202           Crimen    False   
2    3              Pulp Fiction  1994       154           Crimen     True   
3    4              Forrest Gump  1994       142            Drama    False   
4    5           The Dark Knight  2008       152           Acción    False   
5    6                Fight Club  1999       139            Drama     True   
6    7                 Inception  2010       148  Ciencia ficción    False   
7    8                The Matrix  1999       136  Ciencia ficción    False   
8    9  The Shawshank Redemption  1994       142            Drama    False   
9   10              Interstellar  2014       169  Ciencia ficción    False   
10  11                 Gladiator  2000       155           Acción    False   
11  12                   Titanic  1997       195          Romanc

In [18]:
# Tomamos las primeras 100
primeras_100 = datos[:100]
df_peliculas_100 = pd.DataFrame(primeras_100)
print(df_peliculas_100.head())

   id                 titulo   año  duracion  genero  adultos    subtitulos
0   1          The Godfather  1972       175  Crimen    False      [es, en]
1   2  The Godfather Part II  1974       202  Crimen    False      [es, en]
2   3           Pulp Fiction  1994       154  Crimen     True      [es, en]
3   4           Forrest Gump  1994       142   Drama    False  [es, en, fr]
4   5        The Dark Knight  2008       152  Acción    False      [es, en]


In [19]:
# Lista de diccionarios con solo informacion pedida
peliculas = []

for peli in primeras_100:
    peliculas.append({
        "Título": peli["titulo"],
        "Año": peli["año"],
        "Duración (minutos)": peli["duracion"],
        "Género": peli["genero"],
        "Adultos": "sí" if peli["adultos"] else "no"
    })

# Convertimos a DataFrame 
df_pelis = pd.DataFrame(peliculas)
df_pelis.head(10) 


,Título,Año,Duración (minutos),Género,Adultos
0,The Godfather,1972,175,Crimen,no
1,The Godfather Part II,1974,202,Crimen,no
2,Pulp Fiction,1994,154,Crimen,sí
3,Forrest Gump,1994,142,Drama,no
4,The Dark Knight,2008,152,Acción,no
5,Fight Club,1999,139,Drama,sí
6,Inception,2010,148,Ciencia ficción,no
7,The Matrix,1999,136,Ciencia ficción,no
8,The Shawshank Redemption,1994,142,Drama,no
9,Interstellar,2014,169,Ciencia ficción,no


In [20]:
# Guardar el dataframe en CSV sin columna index
df_pelis.to_csv("peliculas.csv", index=False)
print("Archivo peliculas.csv creado con éxito")


Archivo peliculas.csv creado con éxito


#### Fase 2: Creación de la Base de Datos

In [21]:
# Importar conexion a MySQL
import mysql.connector

# Conectarse al servidor MySQL
conexion = mysql.connector.connect(
    host="localhost",       
    user="root",      
    password="AlumnaAdalab"  
)

#cursor para ejecutar comandos SQL desde Python
cursor = conexion.cursor()
print("Conexión exitosa a MySQL")



Conexión exitosa a MySQL


In [22]:
# Nombre de la base de datos
nombre_db = "peliculas_db"

# Crear la base de datos
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_db}")
print(f"Base de datos '{nombre_db}' creada ")

# Seleccionar la base de datos
cursor.execute(f"USE {nombre_db}")
print(f"Usando la base de datos '{nombre_db}'")

Base de datos 'peliculas_db' creada 
Usando la base de datos 'peliculas_db'


In [23]:
# Seleccionar la base de datos
conexion.database = nombre_db
print(f"Usando la base de datos '{nombre_db}'")


Usando la base de datos 'peliculas_db'


In [24]:
# Crear la tabla de películas con todas las columnas 
cursor.execute("""
CREATE TABLE IF NOT EXISTS peliculas (
    id INT AUTO_INCREMENT PRIMARY KEY,
    titulo VARCHAR(255),
    año INT,
    duracion INT,
    genero VARCHAR(100),
    adultos BOOLEAN,
    subtitulos VARCHAR(255)
)
""")
conexion.commit()
print("Tabla creada correctamente")



Tabla creada correctamente


#### Fase 3: Inserción de los Datos en la Base de Datos

In [34]:
# Insertar los datos desde el Json
sql = """
INSERT INTO peliculas (titulo, año, duracion, genero, adultos, subtitulos)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for peli in datos:
    titulo = peli.get("titulo")
    año = peli.get("año")
    duracion = peli.get("duracion")
    genero = peli.get("genero")
    adultos = peli.get("adultos")

    # Subtítulos: puede ser lista o None
    subtitulos = peli.get("subtitulos")
    if isinstance(subtitulos, list):
        subtitulos_str = ", ".join(subtitulos)
    else:
        subtitulos_str = None

    # Ejecutamos el insert
    cursor.execute(sql, (titulo, año, duracion, genero, adultos, subtitulos_str))

conexion.commit()

print("Películas insertadas correctamente en la base de datos.")


Películas insertadas correctamente en la base de datos.


In [35]:
# Verificar que los datos se han guardado

# Selecciona las primeras 10 con fetchall para guardar en Python y poder iterar
cursor.execute("SELECT * FROM peliculas LIMIT 10")
resultado = cursor.fetchall()

# Impimir por fila
for fila in resultado:
    print(fila)


(1, None, None, None, None, None, None)
(2, None, None, None, None, None, None)
(3, None, None, None, None, None, None)
(4, None, None, None, None, None, None)
(5, None, None, None, None, None, None)
(6, None, None, None, None, None, None)
(7, None, None, None, None, None, None)
(8, None, None, None, None, None, None)
(9, None, None, None, None, None, None)
(10, None, None, None, None, None, None)


#### Fase 4: Obtener información a partir de los datos.

¿Cuántas películas tienen una duración superior a 120 minutos?

In [28]:
cursor.execute("SELECT COUNT(*) FROM peliculas WHERE duracion > 120")
resultado = cursor.fetchone()[0]
print("Peliculas con duración > 120 min:", resultado)

Peliculas con duración > 120 min: 0


¿Cuántas películas incluyen subtítulos en español?

In [29]:
cursor.execute("SELECT COUNT(*) FROM peliculas WHERE subtitulos LIKE '%español%'")
resultado = cursor.fetchone()[0]
print("Peliculas con subtitulos en Español", resultado)


Peliculas con subtitulos en Español 0


¿Cuántas películas tienen contenido adulto?

¿Cuál es la película más antigua registrada en la base de datos?

Muestra el promedio de duración de las películas agrupado por género.

¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.

¿Cuál es el año con más películas en la base de datos?

Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier palabra).